# 1 Introduction

## 1.1 Background

Canberra is the capital city of Australia. Canberra approximately has 8 districts with varying number of suburbs. The city is located at the northern end of the Australian Capital Territory (ACT), 280 km (170 mi) south-west of Sydney, and 660 km (410 mi) north-east of Melbourne. With a population over 403,057, it is Australia's largest inland city and the eighth largest city overall.

According to [Australia population site](http://www.population.net.au/canberra-population/), Canberra population can reach 410,199 by end of June of 2019.

In order to accommodate/attract new dwellers, Canberra is expanding its infrastructure by adding new transports and new suburbs.

## 1.2 Problem

With increasing infrastructure and introduction to new suburbs Canberra it becomes a challenge for new residents to figure out which suburb is safe to live and have decent amenities. For the investors it is even more critical to invest in a property that have good suburb profile so that they can see a decent profit in the future. This case study attempts to analyze different suburbs based on various parameters that can help new,old residents and investor to make safe choice for living.

## 1.3 Interest

The case study will be helpful or interesting for:
* the existing residents of Canberra to understand suburb profiles better 
* the families, students or person who is planning to move to Canberra 
* the investors who is looking to invest in Canberra. 

To certain extend it can also attract real estate dealers for determining property price. Being a Canberra resident, I am also intrested to see which suburb is with minimal crime with decent amenities.

## 1.4 Assumptions

* Safety is considered as primary requirement for analyzing suburb profile.
* Based on 1.5km as walking distance, nearby venues are assessed.

## 1.5 Limitation

* Crime stats are only available from 2014 and hence crime rate comparison is made from 2014.
* New suburbs may have limited data to assess.

## 1.6 Case study approach

In order to understand suburb profile, the case study is conducted in two phase:

* Phase 1: Crime rate analysis
* Phase 2: Amenities analysis

Since safety is primary requirement, the study is split into two. First safe districts are identified and then amenities are assessed for safe district suburb.

# Data

* To analyze the crime stats, crime rate data is sourced from [ACT Police website](https://policenews.act.gov.au/crime-statistics-and-data/crime-statistics/download).
* To analyze various amenities, data is sourced from [FourSquare API](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=2ahUKEwj59cm9oc7gAhVKf30KHWVCDSEQFjAAegQIABAC&url=https%3A%2F%2Fdeveloper.foursquare.com%2F&usg=AOvVaw3DzSLcL2tSGGv_ErgcAjEY).
* To plot the map, latitude and longitudes are required. These are obtained from python geopy packages.

The crime data source from ACT police website is in the form of excel sheet and hence data is converted to CSV format for easier modelling. Two types of files were created:

* [District Suburb wise crime total count](https://github.com/radhikakannan/Capstone/blob/master/data/Canberra_crime_stats.csv) - This contains district suburb wise crime total starting from year 2014 till 2017. The data available for each quarter per year.
* [District Suburb wise crimes](https://github.com/radhikakannan/Capstone/blob/master/data/act_suburb_crime_data.csv) - This file contains suburb wise crimes that took place from year 2014 till 2017. The data available for each quarter per year.

Amenities data are in JSON format and hence no data manipulation is done.